In [1]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete
import numpy as np
import random
import os
import pandas as pd
from stable_baselines3 import PPO
from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

# Functie & Environment

In [2]:
def MakeShuffledContainerList(GridRows=3, GridColumns=3, BootAantal=3):
    #The same number of containers on all boats
    number_sample = list(range(1,BootAantal+1))*GridColumns
    #Shuffle the list
    #return random.sample(number_sample, len(number_sample))
    
    return sorted(number_sample, reverse=True)

def MakeRandomContainerList(GridRows=3, GridColumns=3, BootAantal=3):
    Lijst = []
    
    #Fill list with random Containers
    while len(Lijst) != GridRows*GridColumns:
        x = random.randint(1, BootAantal)
        Lijst.append(x)
    
    #Always at least 1 container per boat
    for i in range(1, BootAantal):
        if i not in Lijst:
            return MakeRandomContainerList(GridRows, GridColumns, BootAantal)
    return Lijst

def checkForBoxedInContainer(rowList):
    numberToCheckFor = np.nan
    numberToVerify = np.nan
    reward = 0
    for item in rowList:
        #Skip empty position on grid
        if (item != 0):
            if (np.isnan(numberToCheckFor)):
                numberToCheckFor = item
            if (not np.isnan(numberToVerify)):
                if (item != numberToVerify):
                    #The previous item is boxed in since that item is not equal to where we started and the next item is also different
                    return -5
            #Set number to verify if boxed in
            if ((item != numberToCheckFor) & (np.isnan(numberToVerify))):
                numberToVerify = item

    return reward

In [3]:
class Containers(Env):
    def __init__(self, GridRows, GridColumns, numberOfShips):
        self.GridRows = GridRows
        self.GridColumns = GridColumns
        self.numberOfShips = numberOfShips
        self.state = pd.DataFrame(index=range(GridRows),columns=range(GridColumns)).fillna(0)
        print(type(self.state))
        self.observation_space = Box(low=int(0),high=int(numberOfShips) ,shape=(GridRows,GridColumns), dtype = int)
        self.action_space = MultiDiscrete([numberOfShips,numberOfShips])
        self.list_containers = MakeShuffledContainerList(GridRows, GridColumns, numberOfShips)
    
    def step(self, action):
        #Get a container from the container list
        container = self.list_containers.pop()
        #Get a preview on the position
        preview_cont = self.state.at[action[0], action[1]]
        
        #Reward if the position is empty
        if preview_cont == 0:
            reward = 1
        else:
            reward = -5
        
        #Reward if placed next to the same container
#-------LEFT
        if action[1] == 0:
            #Container Reight the same?
            if self.state.at[action[0], action[1]+1] == container: 
                reward += 1

#-------RIGHT
        elif action[1] ==  len(self.state.head())-1:
            #Container Left the same?
            if self.state.at[action[0], action[1]-1] == container: 
                reward += 1

#-------MIDDLE  
        else:
            #Container Left the same?
            if self.state.at[action[0], action[1]-1] == container:
                reward += 1
            #Container Right the same?
            if self.state.at[action[0], action[1]+1] == container:
                reward += 1
            
            #Container between 2 Containers? 
            #[0,0,X,0,0] Good
            #[1,0,X,0,1] Wrong, Cause it can only place on the sides (Even if it's a 1)
            if not (all(v == 0 for v in self.state.to_numpy()[action[0]][:action[1]]) \
                    or all(v == 0 for v in self.state.to_numpy()[action[0]][action[1]+1:])):
                reward -= 5

        #Placing the container
        self.state.at[action[0],action[1]] = int(container)

        
#-------EXTRA +=; Row fully filled, extra reward
        #First check if it is empty, otherwise the model will constantly place boxes on full rows
        if preview_cont == 0:
            if self.state.eq(self.state.iloc[:,1], axis=0).all(1)[action[0]] == True:
                reward += 5
        
#-------EXTRA +=; The more variation in boat numbers in a row, the worse the reward
        df = self.state.iloc[action[0]].to_numpy()
        if len(set(df[df!=0])) > 2:
            reward -= 2
        
        #When a contianer is being boxed in
        reward += checkForBoxedInContainer(df)
        
        #If the ContainerList is empty, it stops
        if len(self.list_containers) == 0:
            done = True
        else:
            done = False
        
        info = {}
        return self.state.to_numpy(), reward, done, info
    
    #Display the environment
    def render(self, mode='human', close = False):
        display(self.state)
    
    #Set the state (grid) en the containerlist to the default states
    def reset(self):
        self.state = pd.DataFrame(index=range(self.GridRows),columns=range(self.GridColumns)).fillna(0)
        self.list_containers = MakeShuffledContainerList(self.GridRows, self.GridColumns, self.numberOfShips)
        return self.state.to_numpy()

# Test Model

In [4]:
GridRows=4
GridColumns=4
numberOfShips=4
Env = Containers(GridRows, GridColumns, numberOfShips)

# Initialize the state of the environment
state = Env.reset()

# Number of episodes to run the simulation for
episodes = 10

# Loop through the episodes
for episode in range(1, episodes+1):
    # Starting values for the episode
    done = False
    score = 0 
    
    # Loop through the actions within an episode
    while not done:
        # Sample an action from the action space
        action = Env.action_space.sample()

        # Take the action and get the new state, reward, done status, and additional information
        n_state, reward, done, info = Env.step(action)
        
        # Add the reward to the score
        score+=reward
    # Render the environment
    Env.render()
    
    # Print the episode number and score at the end of each episode
    print('Episode:{} Score:{}'.format(episode, score))
    
    # Reset the environment for the next episode
    state = Env.reset()


# Close the environment
Env.close()

<class 'pandas.core.frame.DataFrame'>


,0,1,2,3
0,1,0,0,2
1,0,4,1,4
2,0,4,3,0
3,0,2,3,0


Episode:1 Score:-47


,0,1,2,3
0,0,4,2,3
1,1,0,0,4
2,2,1,4,0
3,0,4,0,3


Episode:2 Score:-32


...

Episode:8 Score:-38


,0,1,2,3
0,2,1,2,3
1,4,0,3,0
2,0,4,0,3
3,0,0,0,4


Episode:9 Score:-36


,0,1,2,3
0,4,0,4,3
1,2,1,4,0
2,1,3,0,0
3,4,2,3,0


Episode:10 Score:-31
